In [ ]:
#Name : Komal Dodiya
#Student ID : U01948955
#Subject : Data Science


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from google.colab import drive
drive.mount('/content/drive')


In [ ]:
month_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/New_A_SN_m_tot_V2.0.csv', sep=';')

In [ ]:
month_df.head()

In [ ]:
month_df.info()


In [ ]:
month_df.describe()


In [ ]:
month_df.isnull().sum()

In [ ]:
month_df.duplicated().sum()

In [ ]:
month_df.corr()

In [ ]:
time_cols = ['year', 'month']
time_col = None

for col in time_cols:
    if col in month_df.columns:
        time_col = col
        break
if time_col is not None:
  print(f'Time column identified as: {time_col}')
  # Further processing based on time_col, e.g.
  # Convert the time column to datetime objects if not already
  if not pd.api.types.is_datetime64_any_dtype(month_df[time_col]):
      try:
          month_df[time_col] = pd.to_datetime(month_df[time_col])
      except ValueError:
          print("Could not convert to datetime automatically. Please check the format of the time column.")
else:
    print("Time column not identified. Please check the dataframe columns.")

In [ ]:
!pip install prophet

from prophet import Prophet


# Prepare the data for Prophet
df_prophet = month_df[['year', 'month', 'total_sunspots']]
df_prophet = df_prophet.rename(columns={'year':'year', 'month':'month', 'total_sunspots':'y'})
df_prophet['ds'] = pd.to_datetime(df_prophet[['year', 'month']].assign(day=1))
df_prophet = df_prophet[['ds', 'y']]

# Create and fit the Prophet model
model = Prophet()
model.fit(df_prophet)

# Reduce the prediction period
future = model.make_future_dataframe(periods=120, freq='MS') # Predicting for 120 months instead of 365

# Predict sunspots
forecast = model.predict(future)

# Print predictions for the specified future time periods
# Adjusting future periods for the reduced prediction horizon
future_periods = [50, 100, 120]
for period in future_periods:
  print(f"\nPrediction for {period} months into the future:")
  print(forecast[['ds', 'yhat']].tail(period))


# Plotting the forecast
fig1 = model.plot(forecast)
plt.title("Sunspots Forecast")
plt.xlabel("Date")
plt.ylabel("Sunspots")
plt.show()


# Plotting the components of the forecast
fig2 = model.plot_components(forecast)
plt.show()

In [ ]:
# Predict sunspots for 1, 6, and 9 months into the future
import pandas as pd # Make sure pandas is imported
from prophet import Prophet

# Recreate df_prophet in this cell:
month_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/New_A_SN_m_tot_V2.0.csv', sep=';')
df_prophet = month_df[['year', 'month', 'total_sunspots']]
df_prophet = df_prophet.rename(columns={'year':'year', 'month':'month', 'total_sunspots':'y'})
df_prophet['ds'] = pd.to_datetime(df_prophet[['year', 'month']].assign(day=1))
df_prophet = df_prophet[['ds', 'y']]

# Create and fit the Prophet model (if not already done)
model = Prophet()
model.fit(df_prophet)

future_periods = [1, 6, 9]  # Months into the future
last_date = df_prophet['ds'].max()



future_periods = [1, 6, 9]  # Months into the future
last_date = df_prophet['ds'].max()

for months in future_periods:
    future_date = last_date + pd.DateOffset(months=months)
    future_df = pd.DataFrame({'ds': [future_date]})
    prediction = model.predict(future_df)
    print(f"\nPrediction for {months} months into the future ({future_date.strftime('%Y-%m-%d')}):")
    print(f"Sunspots: {prediction['yhat'].iloc[0]}")





In [ ]:
# Plotting the forecast for specific future periods
import pandas as pd
from prophet import Prophet

# Recreate df_prophet in this cell to define 'model' and 'forecast':
month_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/New_A_SN_m_tot_V2.0.csv', sep=';')
df_prophet = month_df[['year', 'month', 'total_sunspots']]
df_prophet = df_prophet.rename(columns={'year':'year', 'month':'month', 'total_sunspots':'y'})
df_prophet['ds'] = pd.to_datetime(df_prophet[['year', 'month']].assign(day=1))
df_prophet = df_prophet[['ds', 'y']]

# Create and fit the Prophet model
model = Prophet()
model.fit(df_prophet)

# Reduce the prediction period (if needed)
future = model.make_future_dataframe(periods=120, freq='MS')
forecast = model.predict(future)

# Now proceed with your plotting code
fig3 = model.plot(forecast)
plt.title("Sunspots Forecast with Specific Future Periods")
plt.xlabel("Date")
plt.ylabel("Sunspots")
plt.show()



# Highlight the specific future predictions
for months in future_periods:
    future_date = last_date + pd.DateOffset(months=months)
    plt.axvline(x=future_date, color='Green', linestyle='--', label=f'{months} months' if months == future_periods[0] else '')
    prediction = model.predict(pd.DataFrame({'ds': [future_date]}))
    plt.scatter(future_date, prediction['yhat'].iloc[0], color='red', s=50)

plt.legend() # Show the legend for the highlighted points
plt.show()

In [ ]:
# Predict sunspots for 1, 10, and 20 years into the future
import pandas as pd # Importing pandas to make sure it's available

# Recreate df_prophet to ensure it's defined in this cell's scope
month_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/New_A_SN_m_tot_V2.0.csv', sep=';')
df_prophet = month_df[['year', 'month', 'total_sunspots']]
df_prophet = df_prophet.rename(columns={'year':'year', 'month':'month', 'total_sunspots':'y'})
df_prophet['ds'] = pd.to_datetime(df_prophet[['year', 'month']].assign(day=1))
df_prophet = df_prophet[['ds', 'y']]


# Predict sunspots for 1, 10, and 20 years into the future
future_periods_years = [1, 10, 20]  # Years into the future
last_date = df_prophet['ds'].max()

for years in future_periods_years:
    # Instead of using pd.DateOffset, directly add years to the year component
    future_year = last_date.year + years
    future_date = pd.to_datetime(f"{future_year}-{last_date.month}-{last_date.day}")
    future_df = pd.DataFrame({'ds': [future_date]})

    # Try to handle the OverflowError by catching it and adjusting the prediction range
    try:
        prediction = model.predict(future_df)
        print(f"\nPrediction for {years} years into the future ({future_date.strftime('%Y-%m-%d')}):")
        print(f"Sunspots: {prediction['yhat'].iloc[0]}")
    except OverflowError:
        print(f"\nOverflowError encountered when predicting {years} years into the future.")
        print("Consider reducing the prediction horizon or adjusting the dataset's time range.")

# If OverflowError occurs, try predicting for a shorter period
        # For example, predict for half the original years:
        years = years // 2  # Integer division to get half the years
        future_year = last_date.year + years
        future_date = pd.to_datetime(f"{future_year}-{last_date.month}-{last_date.day}")
        future_df = pd.DataFrame({'ds': [future_date]})
        prediction = model.predict(future_df) # Assuming this shorter prediction works
        print(f"\nPrediction for {years} years into the future instead ({future_date.strftime('%Y-%m-%d')}):")
        print(f"Sunspots: {prediction['yhat'].iloc[0]}")

# Plotting the forecast for specific future periods (in years)
fig4 = model.plot(forecast)
plt.title("Sunspots Forecast with Specific Future Periods (Years)")
plt.xlabel("Date")
plt.ylabel("Sunspots")



In [ ]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score
import pandas as pd # Importing pandas to make sure it's available

# Recreate df_prophet to ensure it's defined in this cell's scope
month_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/New_A_SN_m_tot_V2.0.csv', sep=';')
df_prophet = month_df[['year', 'month', 'total_sunspots']]
df_prophet = df_prophet.rename(columns={'year':'year', 'month':'month', 'total_sunspots':'y'})
df_prophet['ds'] = pd.to_datetime(df_prophet[['year', 'month']].assign(day=1))
df_prophet = df_prophet[['ds', 'y']]


# Prepare the data for Prophet
df_prophet = month_df[['year', 'month', 'total_sunspots']]
df_prophet = df_prophet.rename(columns={'year':'year', 'month':'month', 'total_sunspots':'y'})
df_prophet['ds'] = pd.to_datetime(df_prophet[['year', 'month']].assign(day=1))
df_prophet = df_prophet[['ds', 'y']]

# Create and fit the Prophet model
model = Prophet()
model.fit(df_prophet)

# Reduce the prediction period
future = model.make_future_dataframe(periods=120, freq='MS') # Predicting for 120 months instead of 365

# Predict sunspots
forecast = model.predict(future)

# Print predictions for the specified future time periods
# Adjusting future periods for the reduced prediction horizon
future_periods = [50, 100, 120]
for period in future_periods:
  print(f"\nPrediction for {period} months into the future:")
  print(forecast[['ds', 'yhat']].tail(period))

# Function to evaluate the model
def evaluate_model(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return mae, mape, r2

# Predictions for different time horizons (days)
future_days = [100, 200, 365]
for days in future_days:
  future_date = df_prophet['ds'].max() + pd.DateOffset(days=days)
  future_df = pd.DataFrame({'ds': [future_date]})
  prediction = model.predict(future_df)
  print(f"\nPrediction for {days} days into the future:")
  print(f"Sunspots: {prediction['yhat'].iloc[0]}")

# Forecasting growth options
growth_options = ['logistic', 'linear']
for growth in growth_options:
    model_growth = Prophet(growth=growth)

    # Create a copy of df_prophet to avoid modifying the original dataframe
    df_growth = df_prophet.copy()

    if growth == 'logistic':
        # Assuming 'total_sunspots' has an upper bound of 300
        df_growth['cap'] = 300  # Add the 'cap' column to df_growth for logistic growth
    model_growth.fit(df_growth)  # Fitting the model to the copy
    future_growth = model_growth.make_future_dataframe(periods=365, freq='D')

  # Add the 'cap' column to future_growth for logistic growth
    if growth == 'logistic':
        future_growth['cap'] = 300

    forecast_growth = model_growth.predict(future_growth)
    # Evaluate the model
    y_true = df_prophet['y'].values
    y_pred = forecast_growth['yhat'][:len(y_true)].values
    mae, mape, r2 = evaluate_model(y_true, y_pred)
    print(f"\nGrowth: {growth}")
    print(f"MAE: {mae}, MAPE: {mape}, R2: {r2}")

# Seasonality testing
periods = [30, 90, 365]
fourier_orders = [3, 5, 7]
for period in periods:
    for order in fourier_orders:
        model_seasonal = Prophet()
        model_seasonal.add_seasonality(name='custom_seasonality', period=period, fourier_order=order)
        model_seasonal.fit(df_prophet)
        future_seasonal = model_seasonal.make_future_dataframe(periods=365, freq='D')
        forecast_seasonal = model_seasonal.predict(future_seasonal)

        y_true = df_prophet['y'].values
        y_pred = forecast_seasonal['yhat'][:len(y_true)].values
        mae, mape, r2 = evaluate_model(y_true, y_pred)

        print(f"\nSeasonality - Period: {period}, Order: {order}")
        print(f"MAE: {mae}, MAPE: {mape}, R2: {r2}")


# Trend changepoints tuning
n_changepoints = [10, 20, 30]
changepoint_prior_scales = [0.01, 0.1, 0.5]
for n_cp in n_changepoints:
  for cp_scale in changepoint_prior_scales:
    model_changepoint = Prophet(n_changepoints=n_cp, changepoint_prior_scale=cp_scale)
    model_changepoint.fit(df_prophet)
    future_changepoint = model_changepoint.make_future_dataframe(periods=365, freq='D')
    forecast_changepoint = model_changepoint.predict(future_changepoint)

    y_true = df_prophet['y'].values
    y_pred = forecast_changepoint['yhat'][:len(y_true)].values
    mae, mape, r2 = evaluate_model(y_true, y_pred)

    print(f"\nChangepoints - n_changepoints: {n_cp}, changepoint_prior_scale: {cp_scale}")
    print(f"MAE: {mae}, MAPE: {mape}, R2: {r2}")